# word2vec 개선

CBOW를 어떻게 개선할 수 있을까요?

* 굳이 입력층 matmul 계층을 구현할 필요 없다, 가중치행렬의 특정 행벡터만 잘 뽑으면 그만.
* 은닉층 이후 계산은, 정답 label에 해당하는 친구에만 집중하면 된다.

# embedding layer

In [1]:
# embedding 계층 구현
import numpy as np
W = np.arange(21).reshape(7, 3)
print(W[2])
print(W[5])

[6 7 8]
[15 16 17]


In [2]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

역전파: 앞 층에서 전해진 기울기를, 특정 행에만 더해준다.

In [ ]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        # dout을 dW의 self.idx번째 행에 더해 준다. for 문보다 빠르다.
        return None

## negative sampling

* 원래: 'you'와 'goodbye'를 줄 때, 타깃 단어는 무엇인가요? --> 다중분류
* 변경 이후: 'you'와 'goodbye'를 줄 때, 타깃 단어는 'say(실제 정답)' 인가요? --> 이진분류
* 이 경우 softmax function이 아닌 sigmoid function을 사용한다. (사실 softmax를 2개 class 가지고 하는 것과 차이가 없다.)
* Embedding Dot layer: 은닉층 뉴런과, 출력 측 가중치에서 정답에 해당하는 단어 벡터 간 내적 계산

* 지금까지는 긍정적인 예(정답)에 대해서만 학습 (정답 레이블은 항상 1)
* 부정적인 예(오답)를 입력할 때 결과가 확실하지 않음
* 해결방법-네거티브 샘플링: 적은 수의 부정적 예를 샘플링하고, 긍정적 + 부정적 예의 손실을 더한 값을 최종 손실로 함
* 말뭉치의 통계 데이터 기초로 샘플링 (자주 등장하는 단어 위주로)

In [3]:
# 확률분포를 기초로 한 샘플링
import numpy as np

# 0부터 9까지 숫자 중 하나를 샘플링
print(np.random.choice(10))

1


In [4]:
# 리스트에서 하나만 샘플링
words = ['딸기', '당근', '수박', '참외', '메론']
np.random.choice(words)

'참외'

In [5]:
# 여러 개 샘플링
print(np.random.choice(words, size=5, replace=True)) # 중복있음
print(np.random.choice(words, size=5, replace=False)) # 중복없음

['수박' '수박' '메론' '참외' '참외']
['수박' '참외' '당근' '메론' '딸기']


In [7]:
# 확률분포에 따라 샘플링
p = [.5, .1, .05, .2, .15]
np.random.choice(words, p=p)

'참외'

In [9]:
# word2vec에선 기존 확률분포애 .75를 제곱: 확률이 낮은 단어의 확률을 다소 높임
p = [.7, .29, .01]
new_p = np.power(p, .75)
new_p /= np.sum(new_p)
new_p

array([0.64196878, 0.33150408, 0.02652714])

In [19]:
# UnigramSampler 클래스를 이용
from negative_sampling_layer import UnigramSampler
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = .75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[4 2]
 [1 4]
 [1 3]]


`negative_sampling_layer.py`를 보세요!!

# 개선된 word2vec 학습

`cbow.py`, `train.py`, `eval.py`를 보세요.

* word2vec에서 구한 단어의 분산 표현은, 단어의 복잡한 패턴을 파악할 수 있다.

# 기타 주제

* 전이 학습: 기존 단어의 분산 표현을 이용해 자연어 처리 과제를 수행한다.
  * 단어와 문장을 고정길이 벡터로 변환할 수 있다. -> RNN 등 신경망에 적용할 수 있다.

* 분산 표현이 좋은지 평가할 수 있는 방법?
  * 단어의 유사성, 유추 문제를 활용해 평가